In [1]:
import os
from GetDataLoaders import get_dataloaders, get_short_dataloaders
from architectures.AlexNetFeature import AlexNetFeature
from architectures.AlexNetClassifierModified import AlexNetClassifier
from architectures.LinearTransformationNormModified import LinearTransformationNorm, Normalize
from architectures.ContrastiveLoss import ContrastiveLoss
#from architectures.NCEAverage import NCEAverage
#from architectures.NCELoss import NCELoss
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch import nn
import time
from torch import optim

In [2]:
'''
# we skip the probs for now
gama = 2.0
with open(os.path.join("./PUprobs", 'prob.dat'), 'r') as file_input:
    train_prob_str = file_input.readlines()
    train_prob = [float(i_prob_str.rstrip('\n')) for i_prob_str in train_prob_str]
    print(len(train_prob)/4.0)
    train_weight = [1.0 if 0==i%4 else 1-train_prob[i]**gama for i in range(len(train_prob))]
'''

'\n# we skip the probs for now\ngama = 2.0\nwith open(os.path.join("./PUprobs", \'prob.dat\'), \'r\') as file_input:\n    train_prob_str = file_input.readlines()\n    train_prob = [float(i_prob_str.rstrip(\'\n\')) for i_prob_str in train_prob_str]\n    print(len(train_prob)/4.0)\n    train_weight = [1.0 if 0==i%4 else 1-train_prob[i]**gama for i in range(len(train_prob))]\n'

In [3]:
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"
batch_size = 192
lr = 0.1
LUT_lr = [(90,0.01), (130,0.001), (190,0.0001), (210,0.00001), (230,0.0001), (245,0.00001)]
num_epochs = 245
momentum = 0.9
weight_decay = 5e-4
nesterov = True
Lambdas = {'CE':1.0, 'MSE':1.0, 'NCE':1.0}

loaders = get_dataloaders('imagenet', batch_size=batch_size, num_workers=2)
ndata_train = len(loaders['train_loader'].dataset)
ndata_valid = len(loaders['valid_loader'].dataset)
t = 0.07
m = 4096
gamma = 2

In [4]:
ndata_train, ndata_valid

(80000, 10000)

In [5]:
#from torch.optim.lr_scheduler import ExponentialLR

feature_net = AlexNetFeature().to(device)
classifier_net = AlexNetClassifier().to(device)
transformation_net = LinearTransformationNorm().to(device)

feature_optimizer = optim.SGD(feature_net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
classifier_optimizer = optim.SGD(classifier_net.parameters() ,lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
transformation_optimizer = optim.SGD(transformation_net.parameters() ,lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)

#Schedulers = {'feature':ExponentialLR(feature_optimizer, gamma=1e-6), 'classifier':ExponentialLR(classifier_optimizer, gamma=1e-6), 'transformation':ExponentialLR(transformation_optimizer, gamma=1e-6)}

Networks = {'feature':feature_net, 'classifier':classifier_net, 'transformation':transformation_net}
Optimizers = {'feature':feature_optimizer, 'classifier':classifier_optimizer, 'transformation':transformation_optimizer}

ContrastiveCriterion = ContrastiveLoss(tau=0.1)
#NCE = {'NCEAverage_train': NCEAverage(outputSize=ndata_train, K=m, T=t).cuda(), 'NCEAverage_valid': NCEAverage(outputSize=ndata_valid, K=m, T=t).cuda() , 'NCECriterion_train': NCELoss(ndata_train), 'NCECriterion_valid': NCELoss(ndata_valid)}
Criterions = {'CE': nn.CrossEntropyLoss(reduction='none'), 'MSE':nn.MSELoss() }

In [6]:
#for sample in loaders['train_loader']:
    #print(sample[0].size(), sample[1].size(), sample[2].size())

In [7]:
def adjust_learning_rates(epoch):
    # filter out the networks that are not trainable and that do
    # not have a learning rate Look Up Table (LUT_lr) in their optim_params
    lr = next((lr for (max_epoch, lr) in LUT_lr if max_epoch>epoch), LUT_lr[-1][1])
    for key in Optimizers:
        for g in Optimizers[key].param_groups:
            g['lr'] = lr

def AlexNetDecoupling(batch_idx, batch, PUWeights=None, train=True, accumulation_steps=4):
    data, targets, indices = batch
    
    if train is True:
        Optimizers['feature'].zero_grad()
        Optimizers['classifier'].zero_grad()
        Optimizers['transformation'].zero_grad()
    
    #to cuda
    
    data = data.to(device)
    targets = targets.to(device)
    indices = indices.to(device)

    
    #perform rotations & adjust data shape
    data_90 = torch.flip(torch.transpose(data,2,3),[2])
    data_180 = torch.flip(torch.flip(data,[2]),[3])
    data_270 = torch.transpose(torch.flip(data,[2]),2,3)
    data = torch.stack([data, data_90, data_180, data_270], dim=1)
    batch_size, rotations, channels, height, width = data.size()
    data = data.view([batch_size*rotations, channels, height, width])
    
    #debug for backward
    data.requires_grad = False 
    targets.requires_grad = False
    indices.requires_grad = False
    
    #collect features
    features = Networks['feature'](data)
    features_rot, features_invariance = torch.split(features, 2048, dim=1)
    
    #collect rotation prediction
    pred = Networks['classifier'](features_rot)
    
    
    #average features across 4 rotations
    features_invariance_instance = features_invariance[0::4,:] + features_invariance[1::4,:] + features_invariance[2::4,:] + features_invariance[3::4,:]
    features_invariance_instance = torch.mul(features_invariance_instance, 0.25) #fbar 192x2048
    
    #downsample to 128 & perform normalization of vector
    
    features_128_norm = Networks['transformation'](features_invariance_instance)#192x128
    features_128_norm_0 = Networks['transformation'](features_invariance[0::4,:])
    features_128_norm_90 = Networks['transformation'](features_invariance[1::4,:])
    features_128_norm_180 = Networks['transformation'](features_invariance[2::4,:])
    features_128_norm_270 = Networks['transformation'](features_invariance[3::4,:])
    
    features_128_list = [features_128_norm_0, features_128_norm_90, features_128_norm_180, features_128_norm_270]
    
    with torch.no_grad():
        #stack 192x2048 4 times to be  = 192x4x2048 = 768x2048
        features_invariance_instance_mean = torch.unsqueeze(features_invariance_instance,1).expand(-1,4,-1).clone()
        features_invariance_instance_mean = features_invariance_instance_mean.view(4*len(features_invariance_instance), 2048) #2048
    
    #calculate rotation loss ignore PU for now
    loss_cls_each = Criterions['CE'](pred, targets)
    loss_cls = torch.sum(loss_cls_each)/loss_cls_each.shape[0]
    
    #calculate rotation invariance by MSE
    with torch.no_grad():
        loss_mse = Criterions['MSE'](features_invariance, features_invariance_instance_mean)
    
    #calculate instance loss using NT-xent
    loss_nce = 0.0
    loss_nce = ContrastiveCriterion(features_128_norm_0, features_128_norm) + ContrastiveCriterion(features_128_norm_90, features_128_norm) + ContrastiveCriterion(features_128_norm_180, features_128_norm) + ContrastiveCriterion(features_128_norm_270, features_128_norm)
    
    loss_total = Lambdas['CE']*loss_cls + Lambdas['NCE']*loss_nce
    
    if train is True:
        loss_total.backward()
        Optimizers['feature'].step()
        Optimizers['classifier'].step()
        Optimizers['transformation'].step()
            
    
    #calculate classification accuracy
    pred = pred.argmax(dim=1, keepdim=True)
    correct = pred.eq(targets.view_as(pred)).sum().item()
    
    losses = {'ce':loss_cls.item(), 'mse':loss_mse.item(), 'nce':loss_nce.item(), 'correct':correct}
    
    return losses
    

In [8]:
def train(data_loader, epoch, log_interval=50):
    
    Networks['feature'].train()
    Networks['classifier'].train()
    Networks['transformation'].train()
    
    losses = {'ce':[], 'mse':[], 'nce':[]}
    correct = 0
    train_loss = np.Inf
    train_acc = 0.0
    
    start_time = time.time()
    for batch_idx, sample in enumerate(data_loader):
        
        lossesdict = AlexNetDecoupling(batch_idx, sample, train=True)
        
        losses['ce'].append(lossesdict['ce'])
        losses['mse'].append(lossesdict['mse'])
        losses['nce'].append(lossesdict['nce'])
        correct += lossesdict['correct']
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: CE {:.6f}, MSE {:.6f}, NT-Xent {:.6f}'.format(epoch, batch_idx*len(sample[0]), len(data_loader.dataset), 100. * batch_idx / len(data_loader), lossesdict['ce'], lossesdict['mse'], lossesdict['nce']))
    adjust_learning_rates(epoch)
    end_time = time.time()
    print("Time for epoch pass {}".format(end_time-start_time))
    train_loss = {'ce': float(np.mean(losses['ce'])), 'mse': float(np.mean(losses['mse'])), 'nce':float(np.mean(losses['nce']))}
    train_acc = correct / float(len(data_loader.dataset)*4)
    print('Train set: Average loss: CE {:.4f}, MSE {:.4f}, NT-Xent {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(train_loss['ce'], train_loss['mse'], train_loss['nce'], correct, len(data_loader.dataset)*4, 100.0*train_acc))
    return train_loss, train_acc

def validate(data_loader, epoch, log_interval=50):
    
    Networks['feature'].eval()
    Networks['classifier'].eval()
    Networks['transformation'].eval()
    
    losses = {'ce':[], 'mse':[], 'nce':[]}
    correct = 0
    valid_loss = np.Inf
    valid_acc = 0.0
    start_time = time.time()
    for batch_idx, sample in enumerate(data_loader): 
        with torch.no_grad():
            lossesdict = AlexNetDecoupling(batch_idx, sample, train=False)
        
        losses['ce'].append(lossesdict['ce'])
        losses['mse'].append(lossesdict['mse'])
        losses['nce'].append(lossesdict['nce'])
        correct += lossesdict['correct']
        if batch_idx % log_interval == 0:
            print('Valid Epoch: {} [{}/{} ({:.0f}%)]\tLoss: CE {:.6f}, MSE {:.6f}, NT-Xent {:.6f}'.format(epoch, batch_idx*len(sample[0]), len(data_loader.dataset), 100. * batch_idx / len(data_loader), lossesdict['ce'], lossesdict['mse'], lossesdict['nce']))
    
    end_time = time.time()
    print("Time for epoch pass {}".format(end_time-start_time))
    valid_loss = {'ce': float(np.mean(losses['ce'])), 'mse': float(np.mean(losses['mse'])), 'nce':float(np.mean(losses['nce']))}
    valid_acc = correct / float(len(data_loader.dataset)*4)
    print('Valid set: Average loss: CE {:.4f}, MSE {:.4f}, NT-Xent {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(valid_loss['ce'], valid_loss['mse'], valid_loss['nce'], correct, len(data_loader.dataset)*4, 100.0*valid_acc))
    return valid_loss, valid_acc

def run_main_loop(loaders, num_epochs):
    writer = SummaryWriter('./logs/AlexNet_Unsupervised_Decoupling_Contrastive')
    save_path = "weights/AlexNet_Decoupling_Contrastive.pth"
    best_acc = 0.0
    for epoch in range(num_epochs):
        print("Performing {}th epoch".format(epoch))
        train_loss, train_acc = train(loaders['train_loader'], epoch)
        val_loss, val_acc = validate(loaders['valid_loader'], epoch)
        
        writer.add_scalar('CELoss/train', train_loss['ce'], epoch)
        writer.add_scalar('MSELoss/train', train_loss['mse'], epoch)
        writer.add_scalar('NT-XENTLoss/train', train_loss['nce'], epoch)
        
        writer.add_scalar('CELoss/Valid', val_loss['ce'], epoch)
        writer.add_scalar('MSELoss/Valid', val_loss['mse'], epoch)
        writer.add_scalar('NT-XENTLoss/Valid', val_loss['nce'], epoch)
        
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/Valid', val_acc, epoch)
    
        writer.add_scalar('LR', Optimizers['feature'].param_groups[0]['lr'], epoch)
        
        if val_acc > best_acc :
            best_acc = val_acc
            #save model
            states = {
                'epoch': epoch + 1,
                'feature_net':Networks['feature'].state_dict(),
                'classifier_net':Networks['classifier'].state_dict(),
                'transformation_net':Networks['transformation'].state_dict(),
                'feature_optimizer': Optimizers['feature'].state_dict(),
                'classifier_optimizer': Optimizers['classifier'].state_dict(),
                'transformation_optimizer': Optimizers['transformation'].state_dict(),
                'best_accuracy': best_acc
            }
            torch.save(states, save_path)
            print('Model Saved')

In [9]:
run_main_loop(loaders, num_epochs)

Performing 0th epoch
Train Epoch: 0 [0/80000 (0%)]	Loss: CE 1.394476, MSE 0.209532, NT-Xent 15.919535
Train Epoch: 0 [9600/80000 (12%)]	Loss: CE 1.197274, MSE 0.008705, NT-Xent 23.710211
Train Epoch: 0 [19200/80000 (24%)]	Loss: CE 1.318553, MSE 0.002142, NT-Xent 21.465500
Train Epoch: 0 [28800/80000 (36%)]	Loss: CE 1.278054, MSE 0.000592, NT-Xent 17.542835
Train Epoch: 0 [38400/80000 (48%)]	Loss: CE 1.334713, MSE 0.000129, NT-Xent 13.620476
Train Epoch: 0 [48000/80000 (60%)]	Loss: CE 1.266521, MSE 0.000229, NT-Xent 11.915387
Train Epoch: 0 [57600/80000 (72%)]	Loss: CE 1.283087, MSE 0.000352, NT-Xent 11.861557
Train Epoch: 0 [67200/80000 (84%)]	Loss: CE 1.215621, MSE 0.000527, NT-Xent 11.721905
Train Epoch: 0 [76800/80000 (96%)]	Loss: CE 1.303246, MSE 0.000314, NT-Xent 8.137121
Time for epoch pass 245.0652515888214
Train set: Average loss: CE 1.2866, MSE 0.0033, NT-Xent 15.5817, Accuracy: 129314/320000 (40.4106%)

Valid Epoch: 0 [0/10000 (0%)]	Loss: CE 1.323165, MSE 0.000193, NT-Xent 9.

Valid Epoch: 6 [0/10000 (0%)]	Loss: CE 1.256480, MSE 0.000537, NT-Xent 3.792132
Valid Epoch: 6 [9600/10000 (94%)]	Loss: CE 1.069855, MSE 0.000679, NT-Xent 3.676769
Time for epoch pass 12.899427652359009
Valid set: Average loss: CE 1.2080, MSE 0.0007, NT-Xent 3.5117, Accuracy: 18204/40000 (45.5100%)

Model Saved
Performing 7th epoch
Train Epoch: 7 [0/80000 (0%)]	Loss: CE 1.168001, MSE 0.000693, NT-Xent 2.833296
Train Epoch: 7 [9600/80000 (12%)]	Loss: CE 1.282445, MSE 0.000630, NT-Xent 2.926089
Train Epoch: 7 [19200/80000 (24%)]	Loss: CE 1.221109, MSE 0.000630, NT-Xent 2.938055
Train Epoch: 7 [28800/80000 (36%)]	Loss: CE 1.217887, MSE 0.000558, NT-Xent 2.923594
Train Epoch: 7 [38400/80000 (48%)]	Loss: CE 1.192096, MSE 0.000579, NT-Xent 2.616820
Train Epoch: 7 [48000/80000 (60%)]	Loss: CE 1.224836, MSE 0.000604, NT-Xent 2.614575
Train Epoch: 7 [57600/80000 (72%)]	Loss: CE 1.155788, MSE 0.000628, NT-Xent 2.625103
Train Epoch: 7 [67200/80000 (84%)]	Loss: CE 1.212749, MSE 0.000611, NT-Xent 2

Train Epoch: 13 [57600/80000 (72%)]	Loss: CE 1.175087, MSE 0.000421, NT-Xent 1.472062
Train Epoch: 13 [67200/80000 (84%)]	Loss: CE 1.216305, MSE 0.000440, NT-Xent 1.498981
Train Epoch: 13 [76800/80000 (96%)]	Loss: CE 1.174703, MSE 0.000419, NT-Xent 1.437076
Time for epoch pass 249.0567741394043
Train set: Average loss: CE 1.1531, MSE 0.0004, NT-Xent 1.4463, Accuracy: 156368/320000 (48.8650%)

Valid Epoch: 13 [0/10000 (0%)]	Loss: CE 1.182573, MSE 0.000358, NT-Xent 2.151328
Valid Epoch: 13 [9600/10000 (94%)]	Loss: CE 1.021499, MSE 0.000408, NT-Xent 1.747797
Time for epoch pass 13.312241554260254
Valid set: Average loss: CE 1.1557, MSE 0.0004, NT-Xent 1.7938, Accuracy: 19519/40000 (48.7975%)

Model Saved
Performing 14th epoch
Train Epoch: 14 [0/80000 (0%)]	Loss: CE 1.142243, MSE 0.000395, NT-Xent 1.415633
Train Epoch: 14 [9600/80000 (12%)]	Loss: CE 1.159392, MSE 0.000407, NT-Xent 1.393317
Train Epoch: 14 [19200/80000 (24%)]	Loss: CE 1.169664, MSE 0.000403, NT-Xent 1.347101
Train Epoch: 14

Train Epoch: 20 [0/80000 (0%)]	Loss: CE 1.084026, MSE 0.000352, NT-Xent 0.831013
Train Epoch: 20 [9600/80000 (12%)]	Loss: CE 1.000387, MSE 0.000402, NT-Xent 0.924915
Train Epoch: 20 [19200/80000 (24%)]	Loss: CE 1.047940, MSE 0.000368, NT-Xent 0.949677
Train Epoch: 20 [28800/80000 (36%)]	Loss: CE 1.085425, MSE 0.000374, NT-Xent 0.788123
Train Epoch: 20 [38400/80000 (48%)]	Loss: CE 1.090065, MSE 0.000376, NT-Xent 0.841175
Train Epoch: 20 [48000/80000 (60%)]	Loss: CE 1.112628, MSE 0.000371, NT-Xent 0.876699
Train Epoch: 20 [57600/80000 (72%)]	Loss: CE 1.093094, MSE 0.000398, NT-Xent 0.928065
Train Epoch: 20 [67200/80000 (84%)]	Loss: CE 1.153719, MSE 0.000385, NT-Xent 0.937736
Train Epoch: 20 [76800/80000 (96%)]	Loss: CE 1.124772, MSE 0.000392, NT-Xent 0.867533
Time for epoch pass 248.54260110855103
Train set: Average loss: CE 1.0743, MSE 0.0004, NT-Xent 0.8800, Accuracy: 171527/320000 (53.6022%)

Valid Epoch: 20 [0/10000 (0%)]	Loss: CE 1.155278, MSE 0.000337, NT-Xent 1.380144
Valid Epoch:

Time for epoch pass 250.9131190776825
Train set: Average loss: CE 0.9902, MSE 0.0004, NT-Xent 0.7100, Accuracy: 185921/320000 (58.1003%)

Valid Epoch: 26 [0/10000 (0%)]	Loss: CE 1.131750, MSE 0.000327, NT-Xent 1.128766
Valid Epoch: 26 [9600/10000 (94%)]	Loss: CE 0.898783, MSE 0.000388, NT-Xent 1.034469
Time for epoch pass 13.803361892700195
Valid set: Average loss: CE 1.0136, MSE 0.0004, NT-Xent 0.9969, Accuracy: 22999/40000 (57.4975%)

Model Saved
Performing 27th epoch
Train Epoch: 27 [0/80000 (0%)]	Loss: CE 0.988265, MSE 0.000392, NT-Xent 0.714021
Train Epoch: 27 [9600/80000 (12%)]	Loss: CE 0.927057, MSE 0.000419, NT-Xent 0.704053
Train Epoch: 27 [19200/80000 (24%)]	Loss: CE 0.869052, MSE 0.000408, NT-Xent 0.701940
Train Epoch: 27 [28800/80000 (36%)]	Loss: CE 0.959709, MSE 0.000395, NT-Xent 0.715991
Train Epoch: 27 [38400/80000 (48%)]	Loss: CE 0.961320, MSE 0.000379, NT-Xent 0.666915
Train Epoch: 27 [48000/80000 (60%)]	Loss: CE 0.966509, MSE 0.000424, NT-Xent 0.712278
Train Epoch: 27

...

Time for epoch pass 247.782888174057
Train set: Average loss: CE 0.0125, MSE 0.0001, NT-Xent 0.1914, Accuracy: 319405/320000 (99.8141%)

Valid Epoch: 179 [0/10000 (0%)]	Loss: CE 1.639154, MSE 0.000075, NT-Xent 0.331077
Valid Epoch: 179 [9600/10000 (94%)]	Loss: CE 1.106642, MSE 0.000078, NT-Xent 0.387566
Time for epoch pass 13.714428424835205
Valid set: Average loss: CE 1.3288, MSE 0.0001, NT-Xent 0.3040, Accuracy: 27698/40000 (69.2450%)

Performing 180th epoch
Train Epoch: 180 [0/80000 (0%)]	Loss: CE 0.011668, MSE 0.000080, NT-Xent 0.185582
Train Epoch: 180 [9600/80000 (12%)]	Loss: CE 0.016882, MSE 0.000078, NT-Xent 0.190731
Train Epoch: 180 [19200/80000 (24%)]	Loss: CE 0.016353, MSE 0.000079, NT-Xent 0.195600
Train Epoch: 180 [28800/80000 (36%)]	Loss: CE 0.008793, MSE 0.000081, NT-Xent 0.192216
Train Epoch: 180 [38400/80000 (48%)]	Loss: CE 0.008362, MSE 0.000082, NT-Xent 0.191243
Train Epoch: 180 [48000/80000 (60%)]	Loss: CE 0.007148, MSE 0.000079, NT-Xent 0.184327
Train Epoch: 180 [5

Train Epoch: 186 [38400/80000 (48%)]	Loss: CE 0.008515, MSE 0.000081, NT-Xent 0.195499
Train Epoch: 186 [48000/80000 (60%)]	Loss: CE 0.012450, MSE 0.000078, NT-Xent 0.188918
Train Epoch: 186 [57600/80000 (72%)]	Loss: CE 0.011380, MSE 0.000080, NT-Xent 0.185799
Train Epoch: 186 [67200/80000 (84%)]	Loss: CE 0.016731, MSE 0.000079, NT-Xent 0.190950
Train Epoch: 186 [76800/80000 (96%)]	Loss: CE 0.009302, MSE 0.000081, NT-Xent 0.192433
Time for epoch pass 249.74819350242615
Train set: Average loss: CE 0.0124, MSE 0.0001, NT-Xent 0.1908, Accuracy: 319399/320000 (99.8122%)

Valid Epoch: 186 [0/10000 (0%)]	Loss: CE 1.648664, MSE 0.000075, NT-Xent 0.339967
Valid Epoch: 186 [9600/10000 (94%)]	Loss: CE 1.119376, MSE 0.000078, NT-Xent 0.396261
Time for epoch pass 13.892106056213379
Valid set: Average loss: CE 1.3272, MSE 0.0001, NT-Xent 0.3045, Accuracy: 27682/40000 (69.2050%)

Performing 187th epoch
Train Epoch: 187 [0/80000 (0%)]	Loss: CE 0.015674, MSE 0.000083, NT-Xent 0.189323
Train Epoch: 187

Time for epoch pass 14.530308723449707
Valid set: Average loss: CE 1.3295, MSE 0.0001, NT-Xent 0.3027, Accuracy: 27702/40000 (69.2550%)

Performing 193th epoch
Train Epoch: 193 [0/80000 (0%)]	Loss: CE 0.008754, MSE 0.000080, NT-Xent 0.188425
Train Epoch: 193 [9600/80000 (12%)]	Loss: CE 0.006045, MSE 0.000080, NT-Xent 0.194697
Train Epoch: 193 [19200/80000 (24%)]	Loss: CE 0.011314, MSE 0.000082, NT-Xent 0.187038
Train Epoch: 193 [28800/80000 (36%)]	Loss: CE 0.014316, MSE 0.000084, NT-Xent 0.192200
Train Epoch: 193 [38400/80000 (48%)]	Loss: CE 0.014602, MSE 0.000079, NT-Xent 0.192378
Train Epoch: 193 [48000/80000 (60%)]	Loss: CE 0.008626, MSE 0.000079, NT-Xent 0.191176
Train Epoch: 193 [57600/80000 (72%)]	Loss: CE 0.016157, MSE 0.000077, NT-Xent 0.188098
Train Epoch: 193 [67200/80000 (84%)]	Loss: CE 0.015918, MSE 0.000079, NT-Xent 0.190487
Train Epoch: 193 [76800/80000 (96%)]	Loss: CE 0.008262, MSE 0.000081, NT-Xent 0.199656
Time for epoch pass 250.24615693092346
Train set: Average loss:

Train Epoch: 199 [67200/80000 (84%)]	Loss: CE 0.008507, MSE 0.000079, NT-Xent 0.192628
Train Epoch: 199 [76800/80000 (96%)]	Loss: CE 0.010454, MSE 0.000080, NT-Xent 0.195567
Time for epoch pass 249.95415115356445
Train set: Average loss: CE 0.0123, MSE 0.0001, NT-Xent 0.1906, Accuracy: 319447/320000 (99.8272%)

Valid Epoch: 199 [0/10000 (0%)]	Loss: CE 1.631620, MSE 0.000074, NT-Xent 0.344159
Valid Epoch: 199 [9600/10000 (94%)]	Loss: CE 1.116459, MSE 0.000076, NT-Xent 0.387804
Time for epoch pass 14.400623559951782
Valid set: Average loss: CE 1.3252, MSE 0.0001, NT-Xent 0.3039, Accuracy: 27677/40000 (69.1925%)

Performing 200th epoch
Train Epoch: 200 [0/80000 (0%)]	Loss: CE 0.009392, MSE 0.000079, NT-Xent 0.196830
Train Epoch: 200 [9600/80000 (12%)]	Loss: CE 0.009431, MSE 0.000079, NT-Xent 0.195307
Train Epoch: 200 [19200/80000 (24%)]	Loss: CE 0.008521, MSE 0.000080, NT-Xent 0.187531
Train Epoch: 200 [28800/80000 (36%)]	Loss: CE 0.012954, MSE 0.000081, NT-Xent 0.187068
Train Epoch: 200 

KeyboardInterrupt: 

In [10]:
save_path = "weights/AlexNet_Decoupling_Contrastive200.pth"
states = {
                'feature_net':Networks['feature'].state_dict(),
                'classifier_net':Networks['classifier'].state_dict(),
                'transformation_net':Networks['transformation'].state_dict(),
                'feature_optimizer': Optimizers['feature'].state_dict(),
                'classifier_optimizer': Optimizers['classifier'].state_dict(),
                'transformation_optimizer': Optimizers['transformation'].state_dict(), 
            }
torch.save(states, save_path)